In [1]:
%matplotlib inline

In [ ]:
# This notebook calculates 21-day centered rolling mean for precipitation and temperature from ERA5-Land across Iceland
# and plots the results (Figure 7 in the manuscript)

In [2]:
import xarray as xr
import pandas as pd
from pathlib import Path
import os

In [2]:
# Path to your files
data_path = Path(r"C:/Users/hordurbhe/Documents/Vinna/lamah/lamah_ice/era5_land/data/")

# List of years
years = range(1951, 2024)

# Initialize dictionaries to store daily time series
daily_temp_data = []
daily_precip_data = []

# Loop through the yearly files
for year in years:
    print(f"Processing year {year}...")
    try:
        # Load temperature and precipitation files for the year
        temp_file = data_path / f"ERA5-Land_2m_temperature_{year}.nc"
        precip_file = data_path / f"ERA5-Land_total_precipitation_{year}.nc"

        # Open datasets
        temp_ds = xr.open_dataset(temp_file)
        precip_ds = xr.open_dataset(precip_file)

        # Convert temperature from Kelvin to Celsius
        temp_ds['t2m'] = temp_ds['t2m'] - 273.15

        # Convert precipitation from meters to mm
        precip_ds['tp'] = precip_ds['tp'] * 1000
        
        # Calculate daily means
        daily_temp = temp_ds.resample(time='1D').mean()
        daily_precip = precip_ds.resample(time='1D').last()

        # Calculate spatial mean for each time step
        daily_temp_mean = daily_temp['t2m'].mean(dim=['latitude', 'longitude']).to_dataframe(name='Temperature (°C)')
        daily_precip_mean = daily_precip['tp'].mean(dim=['latitude', 'longitude']).to_dataframe(name='Precipitation (mm)')

        # Append results to lists
        daily_temp_data.append(daily_temp_mean)
        daily_precip_data.append(daily_precip_mean)

    except FileNotFoundError:
        print(f"Missing data for year {year}. Skipping...")

# Combine all years into single DataFrames
daily_temp_df = pd.concat(daily_temp_data)
daily_precip_df = pd.concat(daily_precip_data)

# Save results to CSV
daily_temp_df.to_csv("daily_temperature_iceland.csv")
daily_precip_df.to_csv("daily_precipitation_iceland.csv")

print("Daily time series saved as CSV files.")


Processing year 1951...
Processing year 1952...
Processing year 1953...
Processing year 1954...
Processing year 1955...
Processing year 1956...
Processing year 1957...
Processing year 1958...
Processing year 1959...
Processing year 1960...
Processing year 1961...
Processing year 1962...
Processing year 1963...
Processing year 1964...
Processing year 1965...
Processing year 1966...
Processing year 1967...
Processing year 1968...
Processing year 1969...
Processing year 1970...
Processing year 1971...
Processing year 1972...
Processing year 1973...
Processing year 1974...
Processing year 1975...
Processing year 1976...
Processing year 1977...
Processing year 1978...
Processing year 1979...
Processing year 1980...
Processing year 1981...
Processing year 1982...
Processing year 1983...
Processing year 1984...
Processing year 1985...
Processing year 1986...
Processing year 1987...
Processing year 1988...
Processing year 1989...
Processing year 1990...
Processing year 1991...
Processing year 

In [10]:
# Load results from csv
daily_temp_df = pd.read_csv("daily_temperature_iceland.csv")
daily_temp_df.set_index('time',inplace=True)
daily_temp_df.index = pd.to_datetime(daily_temp_df.index)

daily_precip_df = pd.read_csv("daily_precipitation_iceland.csv")
daily_precip_df.set_index('time',inplace=True)
daily_precip_df.index = pd.to_datetime(daily_precip_df.index)

# Plot figures

In [20]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kendalltau
sns.set()

save_path = Path(r"C:\Users\hordurbhe\OneDrive - Landsvirkjun\Changes in streamflow in Iceland\paper\Figures_Final")

def calculate_and_plot_trends_comparison(df, variable_name, start_year1, end_year1, start_year2, end_year2, window_length=21, use_percentage=False, ax=None):
    """
    Calculate 21-day moving averages, compute daily trends for two periods, and visualize the comparison.
    
    Parameters:
        df (pd.DataFrame): Daily time series with a DatetimeIndex and one column for the variable.
        variable_name (str): Name of the variable (e.g., "Precipitation (mm)" or "Temperature (°C)").
        start_year1 (int): Start year for the first period.
        end_year1 (int): End year for the first period.
        start_year2 (int): Start year for the second period.
        end_year2 (int): End year for the second period.
        window_length (int): Window length for moving average (default: 21 days).
        use_percentage (bool): If True, calculate trends as % per decade; otherwise, use the original unit.
        ax (matplotlib.axes.Axes): Optional. Existing axis to plot on. If None, a new figure is created.
    """
    from seaborn import color_palette
    from matplotlib import rcParams
    sns.set()
    # Set font and figure settings
    rcParams['font.family'] = 'Arial'
    rcParams['font.size'] = 25
    rcParams['axes.titlesize'] = 30
    rcParams['axes.labelsize'] = 25
    rcParams['xtick.labelsize'] = 25
    rcParams['ytick.labelsize'] = 25
    rcParams['legend.fontsize'] = 20
    
    # Use colorblind-safe colors
    colors = color_palette("colorblind", n_colors=2)
    
    def calculate_trends(df, start_year, end_year):
        """
        Helper function to calculate trends for a given period.
        """
        df_period = df[str(start_year):str(end_year)]
        df_ma = df_period.rolling(window=window_length, center=True).mean()
        trends, significance = [], []

        for day in range(1, 366):
            daily_values = df_ma[df_ma.index.dayofyear == day].dropna()
            if len(daily_values) > 0:
                x = np.arange(len(daily_values))
                slope = np.polyfit(x, daily_values.values.flatten(), 1)[0]
                trend = (slope / daily_values.mean().values[0]) * 100 * 10 if use_percentage else slope * 10
                trends.append(trend)
                _, p_value = kendalltau(x, daily_values.values.flatten())
                significance.append(p_value < 0.05)
            else:
                trends.append(np.nan)
                significance.append(False)

        trend_label = "% per decade" if use_percentage else f"{variable_name} per decade"
        return pd.DataFrame({"Day of Year": np.arange(1, 366), "Trend": trends, "Significant": significance, "Trend Label": trend_label})

    # Calculate trends for both periods
    trends_df1 = calculate_trends(df, start_year1, end_year1)
    trends_df2 = calculate_trends(df, start_year2, end_year2)

    # Create a new figure if no axis is provided
    if ax is None:
        fig, ax = plt.subplots(figsize=(14, 7))

    # Plot period 1
    ax.plot(trends_df1["Day of Year"], trends_df1["Trend"], color=colors[0], label=f"Trend {start_year1}-{end_year1}")
    ax.scatter(trends_df1["Day of Year"][trends_df1["Significant"]],
               trends_df1["Trend"][trends_df1["Significant"]],
               color=colors[0], label=f"Significant (p < 0.05) {start_year1}-{end_year1}", zorder=3)

    # Plot period 2
    ax.plot(trends_df2["Day of Year"], trends_df2["Trend"], color=colors[1], label=f"Trend {start_year2}-{end_year2}", linestyle="--")
    ax.scatter(trends_df2["Day of Year"][trends_df2["Significant"]],
               trends_df2["Trend"][trends_df2["Significant"]],
               color=colors[1], label=f"Significant (p < 0.05) {start_year2}-{end_year2}", zorder=3)

    # Add gridlines and labels
    ax.axhline(0, color="black", linestyle="--", linewidth=0.8)
    days_in_months = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]
    month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    ax.set_xticks(days_in_months[:-1])
    ax.set_xticklabels(month_labels)
    ax.grid(linestyle="--", alpha=1)

    # Set titles and labels
    title_insert = "precipitation" if use_percentage else f"temperature"
    trend_ylabel = "% per decade" if use_percentage else f"°C per decade"
    if use_percentage:
        ax.set_title(f"b) Daily trends (21DRM) in {title_insert}, averaged over Iceland")
    else:
        ax.set_title(f"a) Daily trends (21DRM) in {title_insert}, averaged over Iceland")
    ax.set_xlabel("Month")
    #ax.set_ylabel(f"Trend in {variable_name} ({trend_ylabel})")
    ax.set_ylabel(f"{trend_ylabel}")
    ax.set_xlabel(" ")

    ax.legend()

    return trends_df1, trends_df2


In [21]:
# Set font and figure settings
from matplotlib import rcParams

rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 25
rcParams['axes.titlesize'] = 30
rcParams['axes.labelsize'] = 25
rcParams['xtick.labelsize'] = 25
rcParams['ytick.labelsize'] = 25
rcParams['legend.fontsize'] = 20

In [22]:
import os
import matplotlib.pyplot as plt

# Create a 3x1 subplot layout
fig, axes = plt.subplots(3, 1, figsize=(14, 18))

# Plot temperature trends in the first subplot
calculate_and_plot_trends_comparison(
    daily_temp_df,
    variable_name="temperature (°C)",
    start_year1=1973,
    end_year1=2023,
    start_year2=1993,
    end_year2=2023,
    window_length=21,
    use_percentage=False,
    ax=axes[0]
)

# Plot precipitation trends in the second subplot
calculate_and_plot_trends_comparison(
    daily_precip_df,
    variable_name="precipitation (%)",
    start_year1=1973,
    end_year1=2023,
    start_year2=1993,
    end_year2=2023,
    window_length=21,
    use_percentage=True,
    ax=axes[1]
)

# Plot the 21-day moving average for September 20th in the third subplot
def extract_and_plot_series_in_subplot(ax, df, target_date, start_year, end_year, window_length=21):
    """
    Extract the 21-day moving average for a specific date across all years and plot it on a given subplot.
    
    Parameters:
        ax (matplotlib.axes.Axes): Existing axis to plot on.
        df (pd.DataFrame): Time series with a DatetimeIndex.
        target_date (str): Target date in "MM-DD" format (e.g., "09-20").
        start_year (int): Start year for the analysis.
        end_year (int): End year for the analysis.
        window_length (int): Window length for the moving average.
    """
    # Filter data for the specified years
    df_filtered = df[str(start_year):str(end_year)]

    # Calculate the 21-day moving average
    df_ma = df_filtered.rolling(window=window_length, center=True).mean()

    # Extract the values for the target date across all years
    target_series = df_ma[df_ma.index.strftime("%m-%d") == target_date]

    # Plot the extracted series
    ax.plot(target_series.index.year, target_series.values, marker='o', label=f"21DRM on {target_date}")
    ax.set_title(f"c) Precipitation: 21DRM on {target_date} ({start_year}-{end_year})")
    ax.set_xlabel("Year")
    ax.set_ylabel("mm/d")
    ax.grid(linestyle="--", alpha=0.7)
    ax.legend()

# Add the third plot to the last subplot
extract_and_plot_series_in_subplot(
    ax=axes[2],
    df=daily_precip_df,
    target_date="09-20",
    start_year=1973,
    end_year=2023,
    window_length=21
)

# Adjust layout and save the combined figure
plt.tight_layout()

savepath = os.path.join(save_path, 'Figure7_iceland_21DRM_and_sept20.pdf')
plt.savefig(savepath, dpi=300)

savepath = os.path.join(save_path, 'Figure7_iceland_21DRM_and_sept20.png')
plt.savefig(savepath, dpi=300)

plt.show()
